In [2]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import csv
from datetime import datetime

State = "Selangor"
firsturl = "https://www.iproperty.com.my/rent/" + State + "/all-residential/?page=1"
response = urlopen(firsturl)
data = BeautifulSoup(response.read(),'lxml')
case_studies = data.find("ul", {"class":"ant-pagination"})
case_stud_details = case_studies.findAll('li')
totalpage = case_stud_details[-2].text
totalpage

'1244'

In [22]:
#for i in range(1, int(totalpage)+1):
for i in range(100, 110):
    pageurl = "https://www.iproperty.com.my/rent/" + State + "/all-residential/?page=" + str(i)
    page_response = requests.get(pageurl, timeout=5)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    prop_urls = page_content.select ('.jPjrzv')
    
    for w in range(0, 20):
        try:
            prop_url = prop_urls[w].find('a')
            prop_url = prop_url["href"]
            SourceUrl = "https://www.iproperty.com.my" + prop_url

            ind_page_response = urlopen(SourceUrl)
            data = BeautifulSoup(ind_page_response.read(),'lxml')

            prop_names = data.find("div", {"class":"btnpbK"})
            try:
                prop_name = prop_names.find('h2')
                prop_name = prop_name.text
                try:
                    PropertyName, City1, *rest1 = prop_name.split(",")
                except ValueError:
                    PropertyName = prop_name
                    City1 = prop_name
            except AttributeError:
                PropertyName = "N/A"
                City1 = "N/A"

            prop_adds = data.find("div", {"class":"kQbnwj"})
            try:
                prop_add = prop_adds.find('h1')
                prop_add = prop_add.text
                try:
                    PropertyAddress, *rest2 = prop_add.split(",", maxsplit=1)
                    *rest3, Postcode, State2 = prop_add.rsplit(",", maxsplit=2)
                except ValueError:
                    PropertyAddress = prop_add
                    Postcode = "00000"
                    State2 = State
            except AttributeError:
                PropertyAddress = "N/A"
                Postcode = "00000"
                State2 = State

            prop_prices = data.find("div", {"class":"kMZquu"})
            RentalPerMth = prop_prices.text

            prop_builds = data.find("li", {"class":"kiWLOD"})
            prop_build = prop_builds.text
            BuiltUpSize = prop_build[11:]

            prop_types = data.find("div", {"class":"property-attr-propertyType"})
            prop_type = prop_types.text
            PropertyType = prop_type[14:]

            prop_furnishs = data.find("div", {"class":"property-attr-furnishing"})
            prop_furnish = prop_furnishs.text
            Furnishing = prop_furnish[11:] 

            prop_all_details = data.find("ul", {"class":"dusqfP"})
            try:
                prop_details = prop_all_details.findAll("li")
                try:
                    NoOfBedroom = prop_details[0].text        
                except IndexError:
                    NoOfBedroom = "N/A"

                try:
                    NoOfBathroom = prop_details[1].text        
                except IndexError:
                    NoOfBathroom = "N/A"

                try:
                    NoOfParking = prop_details[2].text        
                except IndexError:
                    NoOfParking = "N/A"

            except AttributeError:
                NoOfBedroom = "N/A"
                NoOfBathroom = "N/A"
                NoOfParking = "N/A"

            prop_post_dates = data.find("div", {"class":"property-attr-updatedAt"})
            prop_post_date = prop_post_dates.text
            PostedDate = prop_post_date[12:]

            for alldata in  range (1):
                with open('iproperty4.csv', 'a', newline='') as csv_file:
                    writer = csv.writer(csv_file)
                    writer.writerow([PropertyName, PropertyType, PropertyAddress, City1, State, Postcode, "", "", BuiltUpSize, NoOfBedroom, NoOfBathroom, NoOfParking, Furnishing, RentalPerMth, PostedDate, SourceUrl])
        
        except IndexError:
            continue
                    
print("Writing complete")

Writing complete
